#Setup

In [1]:
!pip install accelerate

In [67]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 713.8 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=96aa07dd77ac899b5c5d355370949f8e7a90eb9778356f6923d585bff57bf3b2
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [57]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)

    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

    predicted_labels = []

    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

#Naampadam Dataset

The _Naampadam_ Dataset is a large dataset for Named Entity Recognition in 11 Indian languages.  _Naampadam_ means "named entity" in Sanskrit.

In [58]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [59]:
# let's now print how the Dataset looks like
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})

In [60]:
raw_datasets.column_names

{'train': ['tokens', 'ner_tags'],
 'test': ['tokens', 'ner_tags'],
 'validation': ['tokens', 'ner_tags']}

In [61]:
# # let's print an instance of dataset
# idx=1000
# rec=raw_datasets['train'][idx]
# for w, t in zip(rec['tokens'],rec['ner_tags']):
#   print('{}\t{}'.format(w,t))


In [8]:
from datasets import Dataset
reduced_train_data = raw_datasets['train'].select(range(20000))

In [9]:
column_names = reduced_train_data.column_names
print(column_names)

features = reduced_train_data.features
print(features)

['tokens', 'ner_tags']

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [62]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [68]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


NameError: name 'features' is not defined

# Training the IndicBERT Model with the dataset

**Load Pre-trained Model**

In [63]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

NameError: name 'num_labels' is not defined

In [ ]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly

model=model.to("cuda")

**Tokenize all texts and align the labels with them**

In [64]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [14]:
train_dataset = reduced_train_data
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset (num_proc=4):   0%|          | 0/20000 [00:00<?, ? examples/s]

In [15]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset (num_proc=4):   0%|          | 0/13460 [00:00<?, ? examples/s]

In [17]:
print(len(train_dataset))
print(len(eval_dataset))

20000

13460


**Create Data Collator, Metrics**

In [65]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [69]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

**Set Training Arguments**

In [70]:
!pip install accelerate
!pip install transformers[torch]

In [22]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)

args=TrainingArguments(
    output_dir='/content/drive/MyDrive/ModelBERT',
    per_device_train_batch_size=16,

    # metric_for_best_model = "f1",
    # evaluation_strategy = "steps",
    # load_best_model_at_end = True,
    per_device_eval_batch_size=8,
    num_train_epochs = 3,
    save_steps = 1875,
    save_safetensors = False

    )


**Training**

In [23]:
# Initialize our Trainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
# args.metric_for_best_model = "f1"
# args.load_best_model_at_end = True
# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = args.save_steps
# args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 

dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)

  warnings.warn(


In [ ]:
trainer.args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_la

In [ ]:
train_result = trainer.train()
metrics = train_result.metrics

Step,Training Loss
500,0.344600
1000,0.303800
1500,0.266500
2000,0.242600
2500,0.230000
3000,0.195500
3500,0.194700


In [24]:
# Getting the predicted valuess

In [71]:
true_tags = raw_datasets['test']['ner_tags']

In [72]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt',padding=True,truncation=True,max_length=128)

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)

    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

    predicted_labels = []

    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

In [73]:
def remove_imbalance_sentences(true_tags,predicted_tags):
      true_balance_tags = []
      predicted_balance_tags = []
      print(type(predicted_tags), type(true_tags))
      for i in range(len(true_tags)):
          if(len(true_tags[i])==len(predicted_tags[i])):
                true_balance_tags.append(true_tags[i])
                predicted_balance_tags.append(predicted_tags[i])
      return true_balance_tags,predicted_balance_tags




In [129]:
check = None

In [169]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_scores(true_tags, predicted_tags):
    # remove tags having  different 
    
#     global check
    true_tags,predicted_tags = remove_imbalance_sentences(true_tags,predicted_tags)

    true_tags_flat = np.concatenate(true_tags)
    predicted_tags_flat = np.concatenate(predicted_tags)

    # Find unique classes
    classes = np.unique(true_tags_flat)

    # Initialize lists to store precision, recall, and F1 scores for each class
    precisions = []
    recalls = []
    f1_scores = []
    
#     check = (true_tags_flat, predicted_tags_flat)
    predicted_tags_flat = [label_to_tags[int_id_to_label[ele]] for ele in predicted_tags_flat]

    # Calculate precision, recall, and F1 score for each class
    for class_id in classes:
#         print(np.array(predicted_tags_flat), class_id)
        true_class = np.array(true_tags_flat) == class_id
        pred_class = np.array(predicted_tags_flat) == class_id

        precision = precision_score(true_class, pred_class, zero_division=1)
        recall = recall_score(true_class, pred_class, zero_division=1)
        f1 = f1_score(true_class, pred_class, zero_division=1)

        # Append scores to respective lists
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

#         print(f"Class {label_to_tags[int_id_to_label[class_id]]}:")
        print(f"Class {class_id}:")
        print(f"  Precision: {precision}")
        print(f"  Recall: {recall}")
        print(f"  F1 Score: {f1}")

    # Calculate macro F1 score
    macro_f1 = sum(f1_scores) / len(f1_scores)

    print("----------------------------")
    print(f"Macro F1 Score: {macro_f1}")

In [76]:
# print(raw_datasets['test']['tokens'])

In [77]:
last_part = "/kaggle/input/modelbert/pytorch/httpswww.kaggle.commodelsnitishk23modelbertframeworkspytorchvariations/1"

In [144]:
import torch
def  prediction_test(path,data):
      model,tokenizer  =  load_model(path)
      predicted_tags = []
      for input_text in data:
          sentence = " ".join(input_text)
          prediction = get_predictions(sentence=sentence,
                                    tokenizer=tokenizer,
                                    model=model
                                    )
         
          last_part = path.split('/')[-1]
          if last_part != "my_model_filtrered_Indic_NEr":
              predicted_tags.append([label_to_int_id[tag] for tag in prediction])
          else:
              print("here")
              new_prediction = []
              for x in prediction:
                  new_prediction.append(x)
              # print(type(new_prediction))

              predicted_tags.append(list(label_to_int_id[label_to_tags_swapped[tag]] for tag in new_prediction))

      return predicted_tags

In [85]:
label_to_tags = {'LABEL_0':'O', 'LABEL_1':'B-PER', 'LABEL_2':'I-PER', 'LABEL_3':'B-ORG', 'LABEL_4':'I-ORG', 'LABEL_5':'B-LOC', 'LABEL_6':'I-LOC', 'LABEL_7':'B-MISC', 'LABEL_8':'I-MISC'}
label_to_int_id = {'LABEL_0':0, 'LABEL_1':1, 'LABEL_2':2, 'LABEL_3':3, 'LABEL_4':4, 'LABEL_5':5, 'LABEL_6':6, 'LABEL_7':7, 'LABEL_8':8}
int_id_to_label = {0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2', 3: 'LABEL_3', 4: 'LABEL_4', 5: 'LABEL_5', 6: 'LABEL_6', 7: 'LABEL_7', 8: 'LABEL_8'}
label_to_tags_swapped = {value: key for key, value in label_to_tags.items()}
# label_to_tags_swapped


In [ ]:
label_to_tags[int_id_to_label[]]

In [80]:
Indic_BERT = "/kaggle/input/modelbert/pytorch/httpswww.kaggle.commodelsnitishk23modelbertframeworkspytorchvariations/1"

all_models_path = {0:Indic_BERT
            }

all_models_names = {0:"Indic_BERT"
              }


In [81]:
# load saved fine-tuned model
import numpy as np
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
def load_model(path):
    tokenizer = AutoTokenizer.from_pretrained(path)
    model = AutoModelForTokenClassification.from_pretrained(path)
    return model,tokenizer


In [114]:
# def one_Sentence(path,sentence):
#       model,tokenizer = load_model(path)
#       prediction = get_predictions(sentence=sentence,
#                                     tokenizer=tokenizer,
#                                     model=model
#                                     )
#       last_part = path.split("/")[-1]
#       if last_part != "my_model_filtrered_Indic_NEr":
#           for index in range(len(sentence.split(' '))):
#               print( sentence.split(' ')[index] + '\t' + label_to_tags[prediction[index]] )
#       else:
#           for index in range(len(sentence.split(' '))):
#               print( sentence.split(' ')[index] + '\t' + prediction[index] )



In [113]:
# sentence = "विश्लेषकों की सिफारिश के मुताबिक, इंद्रप्रस्थ गैस और पीएनबी हाउसिंग फाइनैंस भी अच्छे विकल्प हो सकते हैं।"
# for path in all_models_path.values():
#     last_part = path.split("/")[-1]
#     print(f"\n\nModel : {str(last_part)}\n")
#     one_Sentence(path,sentence)





In [180]:
def each_model_score(data,true_tags_1):
    for path in all_models_path.values():
#         print(f"\n\nModel : {str(last_part)}\n")
        predicted_tags = prediction_test(path,data)
        if last_part!="my_model_filtrered_Indic_NEr":
            calculate_scores(true_tags_1,predicted_tags)
#         return predicted_tags

In [83]:
test_data = raw_datasets['test']['tokens']
test_true_tags = raw_datasets['test']['ner_tags']
len(test_data) , len(test_true_tags)

(867, 867)

In [62]:
each_model_score(test_data)





Model : checkpoint-3750



<class 'list'> <class 'list'>

Class O:

  Precision: 0.9652136699883842

  Recall: 0.971748630516403

  F1 Score: 0.9684701263648633

Class B-PER:

  Precision: 0.8101430429128739

  Recall: 0.7936305732484077

  F1 Score: 0.8018018018018019

Class I-PER:

  Precision: 0.8930099857346647

  Recall: 0.8413978494623656

  F1 Score: 0.8664359861591696

Class B-ORG:

  Precision: 0.710172744721689

  Recall: 0.7370517928286853

  F1 Score: 0.7233626588465297

Class I-ORG:

  Precision: 0.6828282828282828

  Recall: 0.7071129707112971

  F1 Score: 0.6947584789311408

Class B-LOC:

  Precision: 0.7646048109965635

  Recall: 0.7307060755336617

  F1 Score: 0.7472712006717045

Class I-LOC:

  Precision: 0.6474820143884892

  Recall: 0.45226130653266333

  F1 Score: 0.5325443786982249

----------------------------

Macro F1 Score: 0.7620920902104906


In [64]:
# Now load json files of question 1 and question 3 and provide it to the model to calculate the macro-F1-score

**Manually Annotated sentences**

In [175]:
import numpy as np
from datasets import load_dataset, ClassLabel, load_metric, DownloadMode


# Path to your JSON file in Google Drive
file_path = '/kaggle/input/ground-truth-path/Ground Truth Value of Ques 1.json'

# Load dataset from the local JSON file
ground_truth_1_path = load_dataset('json', data_files=file_path)

# Print the information about the loaded dataset
print(ground_truth_1_path)

# import json
# # Load the JSON file
# with open('/content/Ground Truth Value of Ques 1.json', 'r') as file:
#     data = json.load(file)
# ground_truth_1 = []
# for x in data:
#   ground_truth_1.append(x['ner'])

# print(ground_truth_1)

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 25
    })
})


In [176]:
true_tags_1 = ground_truth_1_path['train']['ner_tags']

In [177]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
model_path = "/kaggle/input/modelbert/pytorch/httpswww.kaggle.commodelsnitishk23modelbertframeworkspytorchvariations/1"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/modelbert/pytorch/httpswww.kaggle.commodelsnitishk23modelbertframeworkspytorchvariations/1 and are newly initialized: ['albert.pooler.bias', 'albert.pooler.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [178]:
train_dataset = ground_truth_1_path
train_dataset = train_dataset.map(
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [181]:
each_model_score(ground_truth_1_path['train']['tokens'],ground_truth_1_path['train']['ner_tags'])
# print(ground_truth_1_path)

<class 'list'> <class 'list'>
Class B-LOC:
  Precision: 0.8888888888888888
  Recall: 0.8
  F1 Score: 0.8421052631578948
Class B-MISC:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class B-ORG:
  Precision: 0.7142857142857143
  Recall: 1.0
  F1 Score: 0.8333333333333333
Class B-PER:
  Precision: 0.8333333333333334
  Recall: 0.625
  F1 Score: 0.7142857142857143
Class I-LOC:
  Precision: 1.0
  Recall: 0.2
  F1 Score: 0.33333333333333337
Class I-MISC:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class I-ORG:
  Precision: 1.0
  Recall: 0.8
  F1 Score: 0.888888888888889
Class I-PER:
  Precision: 1.0
  Recall: 0.4
  F1 Score: 0.5714285714285715
Class O:
  Precision: 0.9325842696629213
  Recall: 0.991044776119403
  F1 Score: 0.9609261939218524
----------------------------
Macro F1 Score: 0.5715890331499544


**This is for the ChatGPT's response**

In [98]:
import numpy as np
from datasets import load_dataset, ClassLabel, load_metric, DownloadMode


# Path to your JSON file in Google Drive
file_path = '/kaggle/input/ground-truth-path/Ground Truth Value of Ques 3.json'

# Load dataset from the local JSON file
ground_truth_2_path = load_dataset('json', data_files=file_path)

# Print the information about the loaded dataset
print(ground_truth_2_path)

# import json
# # Load the JSON file
# with open('/content/Ground Truth Value of Ques 3.json', 'r') as file:
#     data = json.load(file)
# ground_truth_2 = []
# for x in data:
#   ground_truth_2.append(x['ner'])

# print(ground_truth_2)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 25
    })
})


In [102]:
true_tags_2 = ground_truth_2_path['train']['ner_tags']
# print(true_tags_2)

In [93]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
model_path = "/kaggle/input/modelbert/pytorch/httpswww.kaggle.commodelsnitishk23modelbertframeworkspytorchvariations/1"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/modelbert/pytorch/httpswww.kaggle.commodelsnitishk23modelbertframeworkspytorchvariations/1 and are newly initialized: ['albert.pooler.bias', 'albert.pooler.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [111]:
# def each_model_score(data):
#     for path in all_models_path.values():
#         print(f"\n\nModel : {str(last_part)}\n")
#         predicted_tags = prediction_test(path,data)
#         if last_part!="my_model_filtrered_Indic_NEr":
#             calculate_scores(true_tags,predicted_tags)

# test_data = train_dataset['test']['tokens']
# test_true_tags = train_datasets['tokens']['ner_tags']
# len(test_data) , len(test_true_tags)


# return each_model_score(test_data)

In [106]:
train_dataset = ground_truth_2_path
train_dataset = train_dataset.map(
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [174]:
each_model_score(ground_truth_2_path['train']['tokens'],ground_truth_1_path['train']['ner_tags'])


<class 'list'> <class 'list'>
Class B-MISC:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class B-PER:
  Precision: 0.5
  Recall: 0.5
  F1 Score: 0.5
Class I-MISC:
  Precision: 1.0
  Recall: 0.0
  F1 Score: 0.0
Class I-PER:
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
Class O:
  Precision: 0.928
  Recall: 0.9747899159663865
  F1 Score: 0.9508196721311476
----------------------------
Macro F1 Score: 0.4901639344262295


In [173]:
ground_truth_2_path

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 25
    })
})